In [1]:
import math
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn import preprocessing

import numpy as np
import pandas as pd

import time
import random
import matplotlib.pyplot as plt
from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from tqdm import tqdm

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

In [3]:
def load_data(directory):
    GSSM = np.loadtxt(directory + '\GSSM_.txt',dtype=np.float32)
    PESSM = np.loadtxt(directory + '\PSSM.txt',dtype=np.float32,delimiter='\t')

    IPE = pd.DataFrame(PESSM).reset_index()
    IG = pd.DataFrame(GSSM).reset_index()
    IPE.rename(columns = {'index':'id'}, inplace = True)
    IG.rename(columns = {'index':'id'}, inplace = True)
    IPE['id'] = IPE['id']
    IG['id'] = IG['id']
    
    return IPE, IG

# def sample(directory, random_seed):
#     all_associations = pd.read_csv(directory + '/all_gpe_pairs.csv')
#     known_associations = all_associations.loc[all_associations['label'] == 1]
#     unknown_associations = all_associations.loc[all_associations['label'] == 0]
#     random_negative = unknown_associations.sample(n=known_associations.shape[0], random_state=random_seed, axis=0)

#     sample_df = known_associations.append(random_negative)
#     sample_df.reset_index(drop=True, inplace=True)

#     return sample_df

def sample(directory, random_seed):
    all_associations = pd.read_csv(directory + '/all_gpe_pairs.csv')
    known_associations = all_associations.loc[all_associations['label'] == 1]
    peco_ids = list(set(known_associations['peco_idx']))
    unknown_associations = all_associations.loc[all_associations['label'] == 0]
    sample_df = known_associations
    for peco_id in peco_ids:
        random_negative = unknown_associations.loc[all_associations['peco_idx'] == peco_id].sample(n=known_associations.loc[all_associations['peco_idx'] == peco_id].shape[0], random_state=random_seed, axis=0, replace=True)

        sample_df = pd.concat([sample_df,random_negative], axis=0)

    sample_df.reset_index(drop=True, inplace=True)

    return sample_df

In [4]:
def performances(y_true, y_pred, y_prob):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()

    pos_acc = tp / sum(y_true)
    neg_acc = tn / (len(y_pred) - sum(y_pred)) # [y_true=0 & y_pred=0] / y_pred=0
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    recall = tp / (tp+fn)
    precision = tp / (tp+fp)
    f1 = 2*precision*recall / (precision+recall)
    
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}|pos_acc={:.4f}|neg_acc={:.4f}'.format(accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc)

In [5]:
def obtain_data(directory, isbalance):

    IPE, IG = load_data(directory)

    if isbalance:
        dtp = sample(directory, random_seed = 1234)
    else:
        dtp = pd.read_csv(directory + '/all_gene_peco_pairs.csv')

    gene_ids = list(set(dtp['gene_idx']))
    peco_ids = list(set(dtp['peco_idx']))
    random.shuffle(gene_ids)
    random.shuffle(peco_ids)
    print('# gene = {} | peco = {}'.format(len(gene_ids), len(peco_ids)))

    gene_test_num = int(len(gene_ids) / 5)
    peco_test_num = int(len(peco_ids) / 5)
    print('# Test: gene = {} | peco = {}'.format(gene_test_num, peco_test_num))    
    
    samples = pd.merge(pd.merge(dtp, IPE, left_on = 'peco_idx', right_on = 'id'), IG, left_on = 'gene_idx', right_on = 'id')
    samples.drop(labels = ['id_x', 'id_y'], axis = 1, inplace = True)
    
    return IPE, IG, dtp, gene_ids, peco_ids, gene_test_num, peco_test_num, samples

In [6]:
def generate_task_Tp_train_test_idx(samples):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1234)

    train_index_all, test_index_all, n = [], [], 0
    train_id_all, test_id_all = [], []
    fold = 0
    for train_idx, test_idx in tqdm(kf.split(samples.iloc[:, 3:])): #train_index与test_index为下标
        print('-------Fold ', fold)
        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)

        train_id_all.append(np.array(dtp.iloc[train_idx][['gene_idx', 'peco_idx']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['gene_idx', 'peco_idx']]))

        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        fold += 1
    return train_index_all, test_index_all, train_id_all, test_id_all

In [7]:
def generate_task_Tg_Tpe_train_test_idx(item, ids, dtp):
    
    test_num = int(len(ids) / 5)
    
    train_index_all, test_index_all = [], []
    train_id_all, test_id_all = [], []
    
    for fold in range(5):
        print('-------Fold ', fold)
        if fold != 4:
            test_ids = ids[fold * test_num : (fold + 1) * test_num]
        else:
            test_ids = ids[fold * test_num :]

        train_ids = list(set(ids) ^ set(test_ids))
        print('# {}: Train = {} | Test = {}'.format(item, len(train_ids), len(test_ids)))

        test_idx = dtp[dtp[item].isin(test_ids)].index.tolist()
        train_idx = dtp[dtp[item].isin(train_ids)].index.tolist()
        random.shuffle(test_idx)
        random.shuffle(train_idx)
        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        assert len(train_idx) + len(test_idx) == len(dtp)

        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)
        
        train_id_all.append(train_ids)
        test_id_all.append(test_ids)
        
    return train_index_all, test_index_all, train_id_all, test_id_all

# 随机森林

In [8]:
def run_rf(train_index_all, test_index_all, samples):
    
    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all):
        print('-----------------------Fold = ', str(fold))

        X = samples.iloc[:, 3:]
        y = samples['label']

        scaler = preprocessing.MinMaxScaler().fit(X.iloc[train_idx,:])
        X = scaler.transform(X)

        x_train, y_train = X[train_idx], y[train_idx]
        x_test, y_test = X[test_idx], y[test_idx]

        clf = RandomForestClassifier(random_state = 19961231)
        clf.fit(x_train, y_train)

        y_train_prob = clf.predict_proba(x_train)
        y_test_prob = clf.predict_proba(x_test)

        y_train_pred = clf.predict(x_train)
        y_test_pred = clf.predict(x_test)

        print('Train:')
        ys_train, metrics_train = performances(y_train, y_train_pred, y_train_prob[:, 1])
        print('Test:')
        ys_test, metrics_test = performances(y_test, y_test_pred, y_test_prob[:, 1])

        fold += 1
        
    return ys_train, metrics_train, ys_test, metrics_test

# DNN

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import *
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve, confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from collections import Counter
from sklearn import preprocessing
from tensorflow.keras import backend as K

In [9]:
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    # Calculates the F score, the weighted harmonic mean of precision and recall.
    #F-beta分数是精度和召回率的加权调和平均值，达到1时为最佳值，0时为最差值
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
        
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0.0:
        return 0.0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def f1(y_true, y_pred):
    # Calculates the f-measure, the harmonic mean of precision and recall.
    return fbeta_score(y_true, y_pred, beta=1)

def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred.reshape(-1)]

In [10]:
def Model(x):
    #Sequential 是实现全连接网络，下面逐层添加网络结构
    model = Sequential()
    model.add(Dense(1024, activation='elu', input_shape=(x.shape[1],)))
    model.add(Dense(512, activation='elu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #优化器为Adam
    #optimizer = Adam(lr = 0.0001)
    model.compile(optimizer = 'Adam', loss = binary_crossentropy, metrics=[binary_accuracy, f1, recall, precision])
    return model

In [13]:
def run_dnn(train_index_all, test_index_all, samples):
    
    #fold = 0
    #for train_idx, test_idx in zip(train_index_all, test_index_all):
        #print('----------------------- Fold = ', str(fold))
    train_idx, test_idx = train_index_all[0], test_index_all[0]
    X = samples.iloc[:, 3:]
    y = samples['label']

    scaler = preprocessing.MinMaxScaler().fit(X.iloc[train_idx,:])
    X = scaler.transform(X)

    x_train, y_train = X[train_idx], y[train_idx]
    x_test, y_test = X[test_idx], y[test_idx]

    model = Model(x_train)
    early_stopping = EarlyStopping(monitor='val_loss', patience = 50)
    model.fit(x_train, y_train, epochs = 500, batch_size = 250, validation_data=(x_test, y_test), callbacks=[early_stopping],verbose = 1)

    y_train_pred, y_test_pred = model.predict(x_train, verbose = 0), model.predict(x_test, verbose = 0)
    y_train_prob, y_test_prob = model.predict_proba(x_train), model.predict_proba(x_test)

    if len(Counter(y_train_pred.reshape(-1))) > 2: 
        y_train_pred = transfer(y_train_pred)
    else:
        print(Counter(y_train_pred.reshape(-1)))
    if len(Counter(y_test_pred.reshape(-1))) > 2: 
        y_test_pred = transfer(y_test_pred)
    else:
        print(Counter(y_test_pred.reshape(-1)))

    performances_train = performances(y_train, y_train_pred, y_train_prob)
    performances_test = performances(y_test, y_test_pred, y_test_prob)

        #fold += 1
    
    return y_train, y_test, y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test

# Run

## RF

In [12]:
directory = 'D:\小麦\MDA-GCNFTG-main\MDA-GCNFTG-main\data'
for isbalance in [True]:
    
    IPE, IG, dtp, gene_ids, peco_ids, gene_test_num, peco_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    #for task in ['Tp', 'Tg', 'Tpe']:
    for task in ['Tp']:   
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tg':
            item = 'gene_idx'
            ids = gene_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tg_Tpe_train_test_idx(item, ids, dtp)

        elif task == 'Tpe':
            item = 'peco_idx'
            ids = peco_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tg_Tpe_train_test_idx(item, ids, dtp)

        ys_train, metrics_train, ys_test, metrics_test = run_rf(train_index_all, test_index_all, samples)
        precision, recall, threshold = precision_recall_curve(ys_test[0], ys_test[2], pos_label=1)
        data1 = pd.DataFrame(zip(precision, recall), columns=['precision','recall']).to_csv(r'D:\小麦\MDA-GCNFTG-main\MDA-GCNFTG-main\评价指标统计图\prc\RF.csv')
#         fpr, tpr, thresholds = roc_curve(ys_test[0], ys_test[2])
#         data = pd.DataFrame(zip(fpr, tpr), columns=['fpr','tpr']).to_csv(r'D:\小麦\MDA-GCNFTG-main\MDA-GCNFTG-main\评价指标统计图\roc\RF.csv',index=False)

5it [00:00, 277.78it/s]

# gene = 11177 | peco = 24
# Test: gene = 2235 | peco = 4
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 37692 | Test = 9424
-------Fold  1
# Pairs: Train = 37693 | Test = 9423
-------Fold  2
# Pairs: Train = 37693 | Test = 9423
-------Fold  3
# Pairs: Train = 37693 | Test = 9423
-------Fold  4
# Pairs: Train = 37693 | Test = 9423
-----------------------Fold =  0


Train:
tn = 18871, fp = 0, fn = 0, tp = 18821
y_pred: 0 = 18871 | 1 = 18821
y_true: 0 = 18871 | 1 = 18821
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000|pos_acc=1.0000|neg_acc=1.0000
Test:
tn = 3649, fp = 1038, fn = 973, tp = 3764
y_pred: 0 = 4622 | 1 = 4802
y_true: 0 = 4687 | 1 = 4737
acc=0.7866|precision=0.7838|recall=0.7946|f1=0.7892|auc=0.9012|aupr=0.9047|pos_acc=0.7946|neg_acc=0.7895
-----------------------Fold =  1
Train:
tn = 18800, fp = 0, fn = 0, tp = 18893
y_pred: 0 = 18800 | 1 = 18893
y_true: 0 = 18800 | 1 = 18893
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000|pos_acc=1.0000|neg_acc=1.0000
Test:
tn = 3627, fp = 1131, fn = 943, tp = 3722
y_pred: 0 = 4570 | 1 = 4853
y_true: 0 = 4758 | 1 = 4665
acc=0.7799|precision=0.7669|recall=0.7979|f1=0.7821|auc=0.8960|aupr=0.8978|pos_acc=0.7979|neg_acc=0.7937
-----------------------Fold =  2
Train:
tn = 18884, fp = 0, fn = 0, tp = 18809
y_pred: 0 = 18884 | 1 = 18809
y_true: 0 = 18884 

## DNN 

In [14]:
directory = 'D:\小麦\MDA-GCNFTG-main\MDA-GCNFTG-main\data'
for isbalance in [True]:
    
    IPE, IG, dtp, gene_ids, peco_ids, gene_test_num, peco_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    #for task in ['Tp', 'Tg', 'Tpe']:
    for task in ['Tp']:   
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tg':
            item = 'gene_idx'
            ids = gene_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tg_Tpe_train_test_idx(item, ids, dtp)

        elif task == 'Tpe':
            item = 'peco_idx'
            ids = peco_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tg_Tpe_train_test_idx(item, ids, dtp)

        y_train, y_test, y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test = run_dnn(train_index_all, test_index_all, samples)
        precision, recall, threshold = precision_recall_curve(y_test, y_test_prob, pos_label=1)
        data1 = pd.DataFrame(zip(precision, recall), columns=['precision','recall']).to_csv(r'D:\小麦\MDA-GCNFTG-main\MDA-GCNFTG-main\评价指标统计图\prc\DNN.csv')
        fpr, tpr, thresholds = roc_curve(y_test, y_test_prob)
        data = pd.DataFrame(zip(fpr, tpr), columns=['fpr','tpr']).to_csv(r'D:\小麦\MDA-GCNFTG-main\MDA-GCNFTG-main\评价指标统计图\roc\DNN.csv',index=False)

5it [00:00, 192.31it/s]

# gene = 11177 | peco = 24
# Test: gene = 2235 | peco = 4
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 37692 | Test = 9424
-------Fold  1
# Pairs: Train = 37693 | Test = 9423
-------Fold  2
# Pairs: Train = 37693 | Test = 9423
-------Fold  3
# Pairs: Train = 37693 | Test = 9423
-------Fold  4
# Pairs: Train = 37693 | Test = 9423
Epoch 1/500


151/151 [==============================] - 2s 12ms/step - loss: 0.6978 - binary_accuracy: 0.5086 - f1: 0.3737 - recall: 0.4573 - precision: 0.4631 - val_loss: 0.6942 - val_binary_accuracy: 0.5184 - val_f1: 0.5531 - val_recall: 0.7451 - val_precision: 0.4890
Epoch 2/500
151/151 [==============================] - 2s 12ms/step - loss: 0.6932 - binary_accuracy: 0.5160 - f1: 0.5023 - recall: 0.5542 - precision: 0.5236 - val_loss: 0.6929 - val_binary_accuracy: 0.5064 - val_f1: 0.4286 - val_recall: 0.3593 - val_precision: 0.5912
Epoch 3/500
151/151 [==============================] - 2s 11ms/step - loss: 0.6934 - binary_accuracy: 0.5149 - f1: 0.4678 - recall: 0.4850 - precision: 0.5215 - val_loss: 0.6929 - val_binary_accuracy: 0.5131 - val_f1: 0.2000 - val_recall: 0.1358 - val_precision: 0.5378
Epoch 4/500
151/151 [==============================] - 2s 11ms/step - loss: 0.6928 - binary_accuracy: 0.5173 - f1: 0.4159 - recall: 0.4287 - precision: 0.5195 - val_loss: 0.6927 - val_binary_accuracy: 0

151/151 [==============================] - 2s 11ms/step - loss: 0.6870 - binary_accuracy: 0.5345 - f1: 0.4405 - recall: 0.3935 - precision: 0.5561 - val_loss: 0.6882 - val_binary_accuracy: 0.5271 - val_f1: 0.3061 - val_recall: 0.2331 - val_precision: 0.5306
Epoch 32/500
151/151 [==============================] - 2s 11ms/step - loss: 0.6853 - binary_accuracy: 0.5437 - f1: 0.4313 - recall: 0.3555 - precision: 0.5727 - val_loss: 0.6838 - val_binary_accuracy: 0.5348 - val_f1: 0.3619 - val_recall: 0.3023 - val_precision: 0.5472
Epoch 33/500
151/151 [==============================] - 2s 11ms/step - loss: 0.6805 - binary_accuracy: 0.5493 - f1: 0.4705 - recall: 0.4101 - precision: 0.5751 - val_loss: 0.6841 - val_binary_accuracy: 0.5342 - val_f1: 0.4398 - val_recall: 0.4283 - val_precision: 0.5254
Epoch 34/500
151/151 [==============================] - 2s 11ms/step - loss: 0.6789 - binary_accuracy: 0.5531 - f1: 0.4581 - recall: 0.3893 - precision: 0.5858 - val_loss: 0.6791 - val_binary_accuracy

151/151 [==============================] - 2s 13ms/step - loss: 0.5275 - binary_accuracy: 0.6739 - f1: 0.6368 - recall: 0.5737 - precision: 0.7203 - val_loss: 0.5843 - val_binary_accuracy: 0.6435 - val_f1: 0.5238 - val_recall: 0.4718 - val_precision: 0.6374
Epoch 62/500
151/151 [==============================] - 2s 12ms/step - loss: 0.5253 - binary_accuracy: 0.6770 - f1: 0.6345 - recall: 0.5682 - precision: 0.7263 - val_loss: 0.5752 - val_binary_accuracy: 0.6419 - val_f1: 0.5446 - val_recall: 0.5203 - val_precision: 0.6103
Epoch 63/500
151/151 [==============================] - 2s 12ms/step - loss: 0.5168 - binary_accuracy: 0.6835 - f1: 0.6489 - recall: 0.5913 - precision: 0.7262 - val_loss: 0.5841 - val_binary_accuracy: 0.6366 - val_f1: 0.5330 - val_recall: 0.5050 - val_precision: 0.6037
Epoch 64/500
151/151 [==============================] - 2s 12ms/step - loss: 0.5194 - binary_accuracy: 0.6802 - f1: 0.6448 - recall: 0.5832 - precision: 0.7263 - val_loss: 0.5802 - val_binary_accuracy

151/151 [==============================] - 2s 11ms/step - loss: 0.4405 - binary_accuracy: 0.7373 - f1: 0.7155 - recall: 0.6662 - precision: 0.7775 - val_loss: 0.6056 - val_binary_accuracy: 0.6646 - val_f1: 0.5868 - val_recall: 0.5763 - val_precision: 0.6287
Epoch 92/500
151/151 [==============================] - 2s 11ms/step - loss: 0.4327 - binary_accuracy: 0.7439 - f1: 0.7243 - recall: 0.6813 - precision: 0.7770 - val_loss: 0.5910 - val_binary_accuracy: 0.6720 - val_f1: 0.5848 - val_recall: 0.5618 - val_precision: 0.6370
Epoch 93/500
151/151 [==============================] - 2s 12ms/step - loss: 0.4287 - binary_accuracy: 0.7461 - f1: 0.7254 - recall: 0.6774 - precision: 0.7856 - val_loss: 0.5999 - val_binary_accuracy: 0.6759 - val_f1: 0.5655 - val_recall: 0.5178 - val_precision: 0.6471
Epoch 94/500
151/151 [==============================] - 2s 11ms/step - loss: 0.4313 - binary_accuracy: 0.7459 - f1: 0.7229 - recall: 0.6689 - precision: 0.7910 - val_loss: 0.6155 - val_binary_accuracy